In [ ]:
import json
import xml.etree.ElementTree as ET

In [ ]:
records_fp = R"C:\Users\Johannes\Downloads\takeout-20220819T193420Z-001\Takeout\Standortverlauf\Records.json"

In [ ]:
records = json.load(open(records_fp, 'r'))

In [ ]:
activity_types = set()
for record in records["locations"]:
    if "activity" in record:
        for a in record["activity"]:
            #activity_types.add(a)
            for t in a["activity"]:
                activity_types.add(t['type'])

In [ ]:
print(activity_types)

In [ ]:
walking_ac = {'WALKING', 'ON_FOOT', 'ON_BICYCLE', 'RUNNING'}
is_moving = False
was_moving = False
tracks = []
new_track = []
for record in records["locations"]:
    if "activity" in record:
        is_moving = False
        for a in record["activity"]:
            #activity_types.add(a)
            for t in a["activity"]:
                if t['type'] in walking_ac:
                    is_moving = True
    if was_moving and not is_moving:
        tracks.append(new_track)
        new_track = []
    if is_moving:
        new_track.append(record)
    was_moving = is_moving
        

In [ ]:
track = tracks[0]
for track in tracks:
    gpx = ET.Element('gpx')
    gpx.set('xmlns', 'http://www.topografix.com/GPX/1/1')
    trk = ET.SubElement(gpx, 'trk')
    name = ET.SubElement(trk, 'name')
    name.text = "Google Standortverlauf"
    trkseg = ET.SubElement(trk, 'trkseg')
    ts_string = track[0]["timestamp"]
    ts_string = ts_string.replace(":", "-")

    for seg in track:
        lat = seg['latitudeE7'] / 1e7
        lon = seg['longitudeE7'] / 1e7
        trkpt = ET.SubElement(trkseg, 'trkpt')
        trkpt.set("lat", str(lat))
        trkpt.set("lon", str(lon))
        time = ET.SubElement(trkpt, 'time')
        time.text = seg["timestamp"]
    gpx_xml = ET.tostring(gpx)
    with open("data/gpx_" + ts_string + ".xml", "wb") as f:
        f.write(gpx_xml)